## Imports

In [1]:
from collections import Counter
from itertools import chain, pairwise

from aoc_utilities import puzzle_input

<h2>--- Day 14: Extended Polymerization ---</h2><p>The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has <a href="https://en.wikipedia.org/wiki/Polymerization" target="_blank">polymerization</a> equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.</p>
<p>The submarine manual contains <span title="HO HO -> OH">instructions</span> for finding the optimal polymer formula; specifically, it offers a <em>polymer template</em> and a list of <em>pair insertion</em> rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.</p>
<p>For example:</p>
<pre><code>NNCB

CH -&gt; B
HH -&gt; N
CB -&gt; H
NH -&gt; C
HB -&gt; C
HC -&gt; B
HN -&gt; C
NN -&gt; C
BH -&gt; H
NC -&gt; B
NB -&gt; B
BN -&gt; B
BB -&gt; N
BC -&gt; B
CC -&gt; N
CN -&gt; C
</code></pre>
<p>The first line is the <em>polymer template</em> - this is the starting point of the process.</p>
<p>The following section defines the <em>pair insertion</em> rules. A rule like <code>AB -&gt; C</code> means that when elements <code>A</code> and <code>B</code> are immediately adjacent, element <code>C</code> should be inserted between them. These insertions all happen simultaneously.</p>
<p>So, starting with the polymer template <code>NNCB</code>, the first step simultaneously considers all three pairs:</p>
<ul>
<li>The first pair (<code>NN</code>) matches the rule <code>NN -&gt; C</code>, so element <code><em>C</em></code> is inserted between the first <code>N</code> and the second <code>N</code>.</li>
<li>The second pair (<code>NC</code>) matches the rule <code>NC -&gt; B</code>, so element <code><em>B</em></code> is inserted between the <code>N</code> and the <code>C</code>.</li>
<li>The third pair (<code>CB</code>) matches the rule <code>CB -&gt; H</code>, so element <code><em>H</em></code> is inserted between the <code>C</code> and the <code>B</code>.</li>
</ul>
<p>Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.</p>
<p>After the first step of this process, the polymer becomes <code>N<em>C</em>N<em>B</em>C<em>H</em>B</code>.</p>
<p>Here are the results of a few steps using the above rules:</p>
<pre><code>Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
</code></pre>
<p>This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, <code>B</code> occurs 1749 times, <code>C</code> occurs 298 times, <code>H</code> occurs 161 times, and <code>N</code> occurs 865 times; taking the quantity of the most common element (<code>B</code>, 1749) and subtracting the quantity of the least common element (<code>H</code>, 161) produces <code>1749 - 161 = <em>1588</em></code>.</p>
<p>Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>


## Solution

In [2]:
def process_polymer(instructions: str, iterations: int):
    polymer_template, pair_insertion_rules = parse_polymer_instructions(instructions)
    pair_counts = Counter(pairwise(polymer_template))

    for _ in range(iterations):
        pair_counts = _process_polymer_step(pair_counts, pair_insertion_rules)

    element_counts = {
        element: sum(
            count for pair, count in pair_counts.items()
            if pair[0] == element
        )
        for element in set(chain.from_iterable(pair_counts))
    }
    element_counts[polymer_template[-1]] += 1

    return element_counts


def parse_polymer_instructions(instructions: str):
    polymer_template, rules_text = instructions.strip().split('\n\n')

    pairs_and_insertions = (
        rule_text.strip().split(' -> ')
        for rule_text in rules_text.split('\n')
    )
    
    pair_insertion_rules = {
        tuple(pair): [(pair[0], insertion), (insertion, pair[1])]
        for pair, insertion in pairs_and_insertions
    }

    return polymer_template, pair_insertion_rules


def _process_polymer_step(pair_counts, pair_insertion_rules):
    pair_counts = pair_counts.copy()
    pairs_to_update = Counter({
        pair_with_rule: pair_counts[pair_with_rule]
        for pair_with_rule in set(pair_counts) & set(pair_insertion_rules)
    })

    for pair_with_rule in pairs_to_update:
        for new_pair in pair_insertion_rules[pair_with_rule]:
            pair_counts[new_pair] += pairs_to_update[pair_with_rule]
    return pair_counts - pairs_to_update

### Testing

In [3]:
example_instructions = '''
    NNCB

    CH -> B
    HH -> N
    CB -> H
    NH -> C
    HB -> C
    HC -> B
    HN -> C
    NN -> C
    BH -> H
    NC -> B
    NB -> B
    BN -> B
    BB -> N
    BC -> B
    CC -> N
    CN -> C
'''

assert process_polymer(example_instructions, 1) == Counter('NCNBCHB')
assert process_polymer(example_instructions, 2) == Counter('NBCCNBBBCBHCB')

example_element_counts = process_polymer(example_instructions, 10).values()
assert max(example_element_counts) - min(example_element_counts) == 1588

### Answer

In [4]:
element_counts = process_polymer(puzzle_input.as_text(day=14), iterations=10)

print(
    'Difference in quantities of most and least frequent elements in polymer:',
    max(element_counts.values()) - min(element_counts.values())
)

Difference in quantities of most and least frequent elements in polymer: 2112


<h2 id="part2">--- Part Two ---</h2><p>The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of <em>40 steps</em> should do it.</p>
<p>In the above example, the most common element is <code>B</code> (occurring <code>2192039569602</code> times) and the least common element is <code>H</code> (occurring <code>3849876073</code> times); subtracting these produces <code><em>2188189693529</em></code>.</p>
<p>Apply <em>40</em> steps of pair insertion to the polymer template and find the most and least common elements in the result. <em>What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?</em></p>


## Solution
Nothing new to see here, folks.

In [5]:
element_counts = process_polymer(puzzle_input.as_text(day=14), iterations=40)

print(
    'Difference in quantities of most and least frequent elements in polymer:',
    max(element_counts.values()) - min(element_counts.values())
)

Difference in quantities of most and least frequent elements in polymer: 3243771149914


## Performance.

In [6]:
%%timeit instructions = puzzle_input.as_text(day=14)

process_polymer(instructions, iterations=40)

5.01 ms ± 40.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
